# Pandas assignment

Cleaning up the below dataframe

In [142]:
import numpy as np
import pandas as pd

In [143]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
 
'Budapest_PaRis', 'Brussels_londOn'], 
 
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085], 
 
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]], 
 
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
 
'12. Air France', '"Swiss Air"']}) 
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [144]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

splitting From_To into From and To columns and dropping the original From_To

In [145]:
df[['From','To']] = df['From_To'].str.split('_',expand=True)
df.drop(columns = 'From_To',inplace=True)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),LoNDon,paris
1,NaN,[],<Air France> (12),MAdrid,miLAN
2,10065.0,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,NaN,[13],12. Air France,Budapest,PaRis
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,londOn


converting From, To column values to lower chars

In [146]:
df['From'] = df['From'].str.lower()
df['To'] = df['To'].str.lower()
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),london,paris
1,NaN,[],<Air France> (12),madrid,milan
2,10065.0,"[24, 43, 87]",(British Airways. ),london,stockholm
3,NaN,[13],12. Air France,budapest,paris
4,10085.0,"[67, 32]","""Swiss Air""",brussels,london


splitting the RecentDelays values of lists into delay_1..n based on max list len and dropping the orginal recent delays

In [147]:
df[['delay_1','delay_2','delay_3']] = pd.DataFrame(df.RecentDelays.values.tolist(), index= df.index)
df.drop(columns = 'RecentDelays',inplace=True)
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3
0,10045.0,KLM(!),london,paris,23.0,47.0,NaN
1,NaN,<Air France> (12),madrid,milan,NaN,NaN,NaN
2,10065.0,(British Airways. ),london,stockholm,24.0,43.0,87.0
3,NaN,12. Air France,budapest,paris,13.0,NaN,NaN
4,10085.0,"""Swiss Air""",brussels,london,67.0,32.0,NaN


fillna(0) for new delays columns

In [148]:
features = ['delay_1','delay_2','delay_3']
for feature in features:
    df[feature] = df[feature].fillna(0)
df    

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3
0,10045.0,KLM(!),london,paris,23.0,47.0,0.0
1,NaN,<Air France> (12),madrid,milan,0.0,0.0,0.0
2,10065.0,(British Airways. ),london,stockholm,24.0,43.0,87.0
3,NaN,12. Air France,budapest,paris,13.0,0.0,0.0
4,10085.0,"""Swiss Air""",brussels,london,67.0,32.0,0.0


Create a new airlines column with alpha airlines names
.str.replace('[^A-Za-z\s]+', '') will replace all chars expect from english aplhabets and spaces

In [149]:
df['Airline'] = df['Airline'].str.replace('[^A-Za-z\s]+', '')
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3
0,10045.0,KLM,london,paris,23.0,47.0,0.0
1,NaN,Air France,madrid,milan,0.0,0.0,0.0
2,10065.0,British Airways,london,stockholm,24.0,43.0,87.0
3,NaN,Air France,budapest,paris,13.0,0.0,0.0
4,10085.0,Swiss Air,brussels,london,67.0,32.0,0.0


Creating a new FlightNumber_filled column - filling NaN values with FFILL + 10

In [150]:
df['FlightNumber_filled'] = df['FlightNumber'].fillna(method='ffill')
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3,FlightNumber_filled
0,10045.0,KLM,london,paris,23.0,47.0,0.0,10045.0
1,NaN,Air France,madrid,milan,0.0,0.0,0.0,10045.0
2,10065.0,British Airways,london,stockholm,24.0,43.0,87.0,10065.0
3,NaN,Air France,budapest,paris,13.0,0.0,0.0,10065.0
4,10085.0,Swiss Air,brussels,london,67.0,32.0,0.0,10085.0


In [151]:
filled_flight_number_series = df.apply(lambda x: x['FlightNumber_filled'] + 10 if pd.isnull(x['FlightNumber']) else x['FlightNumber'], axis=1)
filled_flight_number_series

0    10045.0
1    10055.0
2    10065.0
3    10075.0
4    10085.0
dtype: float64

In [152]:
df['FlightNumber_filled'] = filled_flight_number_series
df.drop(columns = 'FlightNumber', inplace = True)

Final cleaned dataframe

In [153]:
df

,Airline,From,To,delay_1,delay_2,delay_3,FlightNumber_filled
0,KLM,london,paris,23.0,47.0,0.0,10045.0
1,Air France,madrid,milan,0.0,0.0,0.0,10055.0
2,British Airways,london,stockholm,24.0,43.0,87.0,10065.0
3,Air France,budapest,paris,13.0,0.0,0.0,10075.0
4,Swiss Air,brussels,london,67.0,32.0,0.0,10085.0
